# End of week 1 solution

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

After week 2 you'll be able to add a User Interface to this tool, giving you a valuable application.

In [1]:
# imports

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment

load_dotenv()
openai = OpenAI()


In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [6]:
# messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [7]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Certainly! Let’s break down the code snippet you provided:

python
yield from {book.get("author") for book in books if book.get("author")}


### Breakdown of the Code

1. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension. This part of the code is creating a set of unique authors from a collection of `books`.

    - **`for book in books`**: This iterates over each `book` in the `books` iterable (could be a list, tuple, etc.).

    - **`book.get("author")`**: This method tries to get the value associated with the key `"author"` from each `book`. It is a safe way to access dictionary keys; if `"author"` doesn't exist for a particular `book`, it will return `None` rather than raising a `KeyError`.

    - **`if book.get("author")`**: This acts as a filter in the comprehension. It checks if the `author` exists and is truthy (i.e., not `None`, empty string, or similar). If there is no author, that book does not contribute to the resulting set.

    - **Result**: The expression evaluates to a set containing unique author names. Since sets inherently do not allow duplicate entries, even if multiple books have the same author, that author will only appear once in the resulting set.

2. **Yielding Values**:
   - `yield from` is a Python statement used within a generator function. It allows the generator to yield all values from an iterable (in this case, the set created by the comprehension).

   - When the code reaches `yield from`, it effectively delegates the value yielding to the set. This means that, for every unique author in the set, it will yield them one at a time.

### Overall Purpose of the Code

The overall purpose of this code is to iterate through a collection of books, extract the authors from each book, and yield each unique author from the set of authors. By using `yield from`, it produces a generator that emits each author without building a full list or array in memory, making it more memory efficient if the alternative was to return or print all authors at once.

### Example Usage

To illustrate this better, consider the following example:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},  # Duplicate
    {"title": "Book 4"},  # No author
    {"title": "Book 5", "author": "Author C"},
]

def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Using the generator
for author in get_unique_authors(books):
    print(author)


### Output

Author A
Author B
Author C


### Conclusion

In summary, the code efficiently extracts and yields the unique authors from a collection of books, being mindful of missing author information, and utilizes Python's generator capabilities for better memory management, especially useful when dealing with large datasets or collections.

In [9]:
# Get Llama 3.2 to answer

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

**Code Explanation**

This line of Python code uses several advanced features, including generator expressions and dictionary methods.

Here's a breakdown of what each part does:

1. `{...}`: This is called a dictionary comprehension or dictionary expression. It creates an iterator that yields key-value pairs from the dictionary.
2. `book.get("author")`: For each book in the dictionary, this method attempts to retrieve the value associated with the key `"author"`. If the key doesn't exist, it returns `None`.
3. `for book in books if ...`: This is a conditional filter clause that iterates over the items in the `books` dictionary. The condition `if book.get("author")` filters out any dictionaries that don't have an `"author"` key.

So, when we put it all together, this line of code yields an iterator that produces author names from books in the `books` dictionary only if each book has an `"author"` key.

**Why This Code Exists**

The primary purpose of this code is likely to extract author information from a list of dictionaries representing books. In a real-world scenario, you might be working with data from a database or API that returns books as dictionaries.

This line of code helps simplify the process of extracting author names by applying filters to ensure only relevant data is processed. The `yield` keyword is used to create an iterator, which allows the function to remember its state and resume execution where it left off when calling `next()` on it again.

**Example Use Case**

Suppose you have a list of book dictionaries with author information:
```python
books = [
    {"title": "To Kill a Mockingbird", "author": "Harper Lee"},
    {"title": "1984", "author": "George Orwell"},
    {"title": "Pride and Prejudice",},  # No author info
]
```
If you run the code:
```python
authors = yield from {book.get("author") for book in books if book.get("author")}
print(authors)  # Output: ["Harper Lee", "George Orwell"]
```
You'll get a list of author names that have corresponding data in the `books` dictionary.

This code helps maintain data integrity by filtering out any dictionaries with missing `"author"` information, ensuring only relevant data is processed and returned.

# Congratulations!

You could make it better by taking in the question using  
`my_question = input("Please enter your question:")`

And then creating the prompts and making the calls interactively.